# Session 11: Coding Solution

```{contents}

```

In [ ]:
!pip install gymnasium

# Giới thiệu môi trường game Taxi

In [ ]:
import gymnasium as gym # thư viện tạo game
import matplotlib.pyplot as plt

In [ ]:
env = gym.make("Taxi-v3", render_mode="rgb_array")
env.reset() # must be called after creating

In [ ]:
frame = env.render()
plt.imshow(frame)
plt.show()

Môi trường này giả lập việc taxi đón và trả khách.


>Khi episode bắt đầu, taxi sẽ xuất hiện ở vị trí ngẫu nhiên trên bản đồ (grid 5x5).
- Các ô màu thể hiện vị trí mà khách hàng sẽ xuất hiện ngẫu nhiên. Taxi cần chạy đến ô có khách hàng để đón khách.
- Các ô màu cũng thể hiện vị trí khách sạn (nơi trả khách) (có trường hợp địa điểm đón khách cũng là địa điểm trả khách), taxi cần chạy đến ô khách sạn trả khách.
>
>Episode sẽ kết thúc khi **taxi thực hiện hành động trả khách**.

Nếu ảnh game khó hình dung grid, bạn có thể tham khảo hình dưới
```
R, G, Y, B là các điểm đón/trả khách
Các ô rỗng là vị trí taxi có thể đi
Kí tự | và : ám chỉ lane đường

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
```


**Agent có thể làm 6 hành động**
- `0` Đi xuống
- `1` Đi lên
- `2` Sang phải
- `3` Sang trái
- `4` Đón khách
- `5` Trả khách

**Reward**
- `+20` trả khách thành công
- `-10` đón hoặc trả khách sai địa điểm
- `-1` nếu không thuộc 2 trường hợp trên (mỗi lần taxi di chuyển)


Cùng tìm hiểu hệ thống các `State` trong môi trường này nhé

In [ ]:
first_state = env.reset()
plt.imshow(env.render())
plt.show()
print()
print(f"First state: {first_state}")

In [ ]:
print(f"Action Space: {env.action_space}")
print(f"State Space: {env.observation_space}")

Game có tổng cộng 500 state: 25 vị trí taxi, 5 vị trí của khách (có 1 vị trí là "in taxi") 4 vị trí khách sạn → `Q Table` là 1 ma trận có shape `(500, 6)` (500 state, 6 action)

State đầu tiên (khi gọi `reset()` ) mà game trả về là 1 `tuple`. Bạn chỉ cần quan tâm con số đầu tiên trong `tuple` này thôi. Con số này là 1 số nguyên biểu diễn state hiện tại theo công thức sau
```
((taxi_row_location * 5 + taxi_col_location) * 5 + passenger_location) * 4 + destination
```

Trong đó, `passenger_location` và `destination` được đánh số như sau

`passenger_locations`
- `0` Red
- `1` Green
- `2` Yellow
- `3` Blue
- `4` In taxi

`destinations`
- `0` Red
- `1` Green
- `2` Yellow
- `3` Blue

Vì mỗi lần chạy sẽ khác nhau, bạn hãy tự kiểm tra lại công thức trên nhé. Ở lần chạy của giảng viên, state là `461` với ảnh render sau

![](https://i.imgur.com/UWEw5um.png)

Số `461` gồm:
- `taxi_row_location = 4` (góc tọa độ `0, 0` ở trên bên trái)
- `taxi_col_location = 3`
- `passenger_location = 0` (ô màu đỏ)
- `destination = 1` (ô màu xanh)

```
461 = ((taxi_row_location * 5 + taxi_col_location) * 5 + passenger_location) * 4 + destination
```


Hàm `step(action)` của môi trường sẽ trả về lần lượt các thông tin sau
- `next_state` số nguyên biểu diễn state tiếp theo
- `reward`
- `terminal` gameover hay không
- `truncate` KHÔNG DÙNG. Game được cài đặt sẽ kết thúc sau 200 steps (tương tự  biến `max_steps` trong bài lab, phần bên dưới ta sẽ thực hành q-learning với `max_steps=99`)
- `info` KHÔNG DÙNG


In [ ]:
# đối với first_state lấy ra từ env.reset()
# thì first_state là tuple
# số nguyên biểu diễn state là phần từ đầu tiên trong tuple
first_state = env.reset()
print(first_state[0])
# to render
# plt.imshow(env.render())
# plt.show()

# next_state khi gọi step là 1 số nguyên luôn chứ không phải tuple
next_state, reward, terminal, _, _ = env.step(1)
# to render
# plt.imshow(env.render())
# plt.show()
print(next_state, reward, terminal)

# Q-Learning



#### TODO 1

Cho sẵn các thông số sau
```python
import numpy as np

q_table = np.zeros(shape=(env.observation_space.n, env.action_space.n)) # 500, 6

episodes = 10000  # số lượng episode cần huấn luyện
learning_rate = 0.1  # learning rate
max_steps = 99  # Số step tối đa để gameover
gamma = 1  # discount factor

# thông số của thuật toán epsilon greedy with decay
# epsilon sau khi kết thúc episode = epsilon * epsilon_decay_rate
initial_epsilon = 1.0  
min_epsilon = 0.001
epsilon_decay_rate = 0.9

# Evaluation
# Mỗi 1000 episode, bạn cho taxi chơi theo greedy-policy 50 lần và thống kê reward
evaluate_at_episode = 1000
num_eval = 50
```
Dùng **thuật toán Q-Learning** để giải bài toán này.

Bạn có thể tham khảo gợi ý bên dưới, khuyến khích học viên làm theo để tự suy nghĩ xem nên thiết kế các hàm như thế nào. Khi viết hàm, bạn nên hạn chế gọi biến GLOBAL mà nên truyền các biến này vào làm tham số của hàm

**Gợi ý**

Để code đẹp, bạn nên viết các hàm hỗ trợ sau:
- `follow_policy` dùng `argmax` trên `q_table` để đi theo policy tốt nhất
- `epsilon_greedy` nếu các action có cùng `q_values`, bạn có thể dùng argmax cho đơn giản (gọi hàm `follow_policy` bên trên)
- `update_q`
- `decay_epsilon`
- `train_one_episode` hàm để huấn luyện Agent 1 lần
- `evaluate_agent` hàm để đánh giá Agent (xem thông số bên trên)

Sau khi viết xong các hàm trên, training loop của bạn nhìn sẽ rất đẹp
```python
# dùng để render progress bar khi train
from tqdm.notebook import tqdm
...
reward_history = []
for i in tqdm(range(episode)):
  average_reward = train_one_episode(...)
  # có thể vẽ chart xem sự thay đổi của reward sau khi train
  reward_history.append(average_reward)
  current_epsilon = decay_epsilon(...)
  # evaluate
  if i == 0 or (i+1) % evaluate_at_episode == 0:
    eval_average_reward, num_loss = evaluate_agent(...)
    print(f"Evaluate after episode {i}")
    print(f"   - Average reward: {eval_average_reward}")
    print(f"   - Loss          : {num_loss}/50")
```

In [ ]:
# YOUR SOLUTION


## Xem Agent chơi sau khi Train

In [ ]:
import cv2

frames = []
# chơi 10 episode và thu thập các frame ảnh
for i in range(10):
  state = env.reset()[0]
  # lưu lại frame ảnh
  # resize để giảm kích thước cho nhẹ
  frame = env.render()
  frame = env.render()
  new_size = int(frame.shape[1] * 0.5), int(frame.shape[0] * 0.5)
  frame = cv2.resize(frame, new_size)
  frames.append(frame)
  step = 0
  while True:
    action = np.argmax(q_table[state])
    state, reward, terminal, _, _ = env.step(action)
    frame = env.render()
    new_size = int(frame.shape[1] * 0.5), int(frame.shape[0] * 0.5)
    frame = cv2.resize(frame, new_size)
    frames.append(frame)
    step += 1
    if terminal or step > 200:
      break

Chạy cell bên dưới để tạo animation, bạn cần xóa animation này trước khi nộp bài

In [ ]:
import matplotlib.animation as anim
from IPython.display import HTML
import matplotlib.pyplot as plt

limit_frames = frames[:100]
plt.figure(figsize=(limit_frames[0].shape[1] / 18.0, limit_frames[0].shape[0] / 18.0), dpi = 72)
patch = plt.imshow(limit_frames[0])
plt.axis('off')
animate = lambda i: patch.set_data(limit_frames[i])
ani = anim.FuncAnimation(plt.gcf(), animate, frames=len(limit_frames), interval = 300)
HTML(ani.to_jshtml())